# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = "output_data/cities.csv"
weather_df = pd.read_csv(file)
del weather_df['Unnamed: 0']
weather_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Keats,0,AU,1620955205,26,-14.2500,129.5500,81.99,13.18
1,Poum,23,NC,1620955206,74,-20.2333,164.0167,77.38,16.04
2,Hilo,90,US,1620955056,65,19.7297,-155.0900,78.80,9.22
3,Khandyga,100,RU,1620955208,31,62.6667,135.6000,48.79,5.48
4,Port Alfred,0,ZA,1620955210,82,-33.5906,26.8910,59.74,3.78
...,...,...,...,...,...,...,...,...,...
549,Sairang,40,IN,1620955879,78,23.8000,92.6667,75.20,8.05
550,Tezu,100,IN,1620955613,98,27.9167,96.1667,66.25,2.64
551,Quesnel,90,CA,1620955882,51,52.9999,-122.5029,62.60,8.05
552,Umm Kaddadah,11,SD,1620955655,44,13.6017,26.6876,80.69,4.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

#Get lat and lng and store them in a location
location = weather_df[['Lat','Lng']].astype(float)

#Get the Humidity levels and store them in humidity_levels
humidity_levels = weather_df['Humidity']

In [20]:
# Create Humidity Heatmap gmaps figure with humidy as the weights
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity_levels, 
                                 dissipating=False, max_intensity=250,
                                 point_radius = 4)
#Add the heat layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrowing down Temp >70 & <80
ideal_temp = weather_df.loc[(weather_df['Max Temp'] >= 70) & (weather_df['Max Temp'] <= 80)]
#Wind <10
ideal_wind = ideal_temp.loc[(weather_df['Wind Speed'] < 10)]
#No Clouds
ideal_weather = ideal_wind.loc[(weather_df['Cloudiness'] == 0)]
#Dropping andy NA data if any
ideal_weather.dropna()
#checking how many locations have those Ideal Weather Condition specified
count = len(ideal_weather)
print(f"** There are {count} locations that have those Ideal Weather Conditions **")
#Show the DataFrame with Ideal Weather Condition
ideal_weather

** There are 10 locations that have those Ideal Weather Conditions **


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
61,Blackwater,0,AU,1620955279,23,-23.5833,148.8833,74.82,8.23
77,Saint-Philippe,0,RE,1620955298,68,-21.3585,55.7679,73.40,6.91
123,Saint-Pierre,0,RE,1620955355,68,-21.3393,55.4781,71.60,6.91
156,Moro,0,PK,1620955395,65,26.6646,68.0016,78.80,4.61
172,Praia,0,CV,1620955413,68,14.9215,-23.5087,73.40,6.91
204,Vao,0,NC,1620955453,82,-22.6667,167.4833,76.08,9.95
279,Bundaberg,0,AU,1620955402,35,-24.8500,152.3500,75.99,9.22
427,Umluj,0,SA,1620955726,58,25.0213,37.2685,79.47,8.75
465,Walvis Bay,0,NaN,1620955772,26,-22.9575,14.5053,71.60,8.05
518,Saint-Joseph,0,RE,1620955840,68,-21.3667,55.6167,71.60,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Storing Ideal weather data to new Hotel DataFrame and start with fresh index
hotel_df = ideal_weather.reset_index(drop=True)

#Adding new column "Hotel Name"
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Blackwater,0,AU,1620955279,23,-23.5833,148.8833,74.82,8.23,
1,Saint-Philippe,0,RE,1620955298,68,-21.3585,55.7679,73.40,6.91,
2,Saint-Pierre,0,RE,1620955355,68,-21.3393,55.4781,71.60,6.91,
3,Moro,0,PK,1620955395,65,26.6646,68.0016,78.80,4.61,
4,Praia,0,CV,1620955413,68,14.9215,-23.5087,73.40,6.91,
5,Vao,0,NC,1620955453,82,-22.6667,167.4833,76.08,9.95,
6,Bundaberg,0,AU,1620955402,35,-24.8500,152.3500,75.99,9.22,
7,Umluj,0,SA,1620955726,58,25.0213,37.2685,79.47,8.75,
8,Walvis Bay,0,NaN,1620955772,26,-22.9575,14.5053,71.60,8.05,
9,Saint-Joseph,0,RE,1620955840,68,-21.3667,55.6167,71.60,6.91,


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration passing in the Lat and Lng
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
   
    #convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.5816891,
                    "lng": 148.8870997
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.58043062010728,
                        "lng": 148.8886849298927
                    },
                    "southwest": {
                        "lat": -23.58313027989272,
                        "lng": 148.8859852701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "The Capricorn Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 383,
                    "html_attributions": [
                        

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ3Gnqj-U-pW_LEdL7-w8-MqdA-R2vUk7fSkapDgHhQE5twY0M05rQN1IYDci1KHhRkze3u4Np_RQIF9CPGFm8LfIP-d0A-GMzxwKn1KRJP28bvl1cyOWHPjpKXZi5-NsyvXjjnIS8VVYd36143PBduB6BETrGGXguXM9tl80dxUV36ydFr85_2Raok8XDWuY3TBmMpxWpioRyneI0DClLW52YDNthnxniSEaLGRm9OoWWSeyitlOwb2NUe9hRUoFbv57MEWGT69MuhaZnKNadJ1zmXCvkl54IIjCycVhP0smPmvwMlHCRdW5r_NP5M7b3F0cAGV-vK82Y7WswxKfyR99URvkItIvlvl_gECPWeRuSjNbTsfajccf3goTnBCKxm1N0KnkYgy3np7pYv1GOp1lNDXsELkW5SsblqTlteSS-ZCBTD-pIX2eibSlnv",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.183965,
                    "lng": 55.29152699999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.18261482010728,
                        "lng": 55.29291202989273
                    },
                    "southwest": {
                        "lat": -21.

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKwG9BbiAav8QnN0TFKrNBrekmE0Yz6Zn7vrshUtOfRVzfjzRY52qBYLlJsj83Iw_SrX1JsNLRjGQyBY77PYdyrnUsBwdV_nrAzsTReKmk-kevuWmPPuppLxMT3s1BaHy5qrDgVtrOvHbc-9cy1etLkza3loKVgZBAmUt7y7pjQ4_EK2nsI9Mk_G_SHNXJo2ibhoVaUEM5vhWHQS_nE-ya8LvcCm0DEP8gpKJ8BOLD0oPD3dFezrYpV2bUp3fgSAadD9vLiNxVdYNU-pJz32DJE791IrOileWNlFXSUTGwQZRjHtcuTbk8eXYSGxxNT44ZID3EpZFxsidKoc0me2O6JPVKat6oC3yn0lRcX_T_3oOZYdGIMTUFkD2kkfoVDe75rh86qcnBJTAY14Tp6uvi_rQlMLcloqJoxigRBSjaXE3uXd6kHs6AHqLocNfdE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.14818,
                    "lng": 55.27768
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.14686767010728,
                        "lng": 55.27904012989272
                    },
                    "southwest": {
                        "lat": -21.1495673298

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.7285454,
                    "lng": 67.75232609999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.72989297989272,
                        "lng": 67.75350417989272
                    },
                    "southwest": {
                        "lat": 26.72719332010728,
                        "lng": 67.75080452010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bhittai Hotel",
            "photos": [
                {
                    "height": 2160,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/107449366638189340885\">Farhan Ba

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLHKZElfLlh32bFuK-uSS55yAYsxW8KXKb--_SyGvQ9jn-tA85EeIOUhrCIahIs1w2WV8VoKZuFmFsL8q-yDv7i7aQJqT4b1DBFhYQL2zeSxAtIaLku8jN2gB9fz4h1zA7w6I7OZNow15JmkQIOZtQt82BCU2_D4CXmRp7pQhFyucxUqDhAHYjs7XUoOFc1x3s-oTMuhjYFxSHwEPmtNoOmR6_8zJHwgBVg9QOMH6PJZp0XAMGoJOL5p5uWXDI1qoprEG-VXakTfHIpfTg0kTB7K53gCjqitF6HOEEHdfg1pZKgWNtJvFuQasbrRa66rVs_jGg2Cf99mQMkkGsUpCGEBhQH0yWmsy5yGdbSfUVYMGcTg1RifD1fZRLnA7nZdgfD4hkXDjB386uhsarS5hSgia7AL5Yin00qIAsbH1WmqlYurj63GFvrWP2g-4XT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.9029006,
                    "lng": -23.5127686
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.90445982989272,
                        "lng": -23.51124162010727
                    },
                    "southwest": {
                        "lat": 14.9017601

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.6644075,
                    "lng": 167.4485915
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.66324597010728,
                        "lng": 167.4500809298927
                    },
                    "southwest": {
                        "lat": -22.66594562989272,
                        "lng": 167.4473812701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "OURE TERA beach resort",
            "photos": [
                {
                    "height": 2988,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/107455838709836963932\">syl

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIA-FYEh3HgJqLNsipIS3jhKSuRGN0BoUqqPpdDPWk3Xq8zZumBdp7Mz9YVRJ-WmVP-9ugIThyxLjmotLU9RJHIrr8Yqf3aZtSbF_L29oaowzt6NMeYt_KPB-i-2kzjbO3VRWKmDpZdm-ub6PcBP-6Do-4BMjGLBMnfE5nYjU9UU70X3bdx6op0gI8MZwnlZ5WY35iIy51rXeClneUDMgtmdpkpd9oQCme6N8RvS8zxiHsgbRMpAkplejzQ9ClDwjAVtSdqLj6PSYOr6Hajz0TmcUFmOTHPZerY9cQI7bEo4qE4Pyb-1Wuioxf5fGS72prLs0G_fnTpcesM9e6fdl_o5KYfwchWypYPbw6_pbwyo_8e7rIy_ldFl1o2fP4NUYg1dKSWiTLllelLrATPDc61e1IomAqPOjmNifMW_9UwccRaxKqDdqTTSsI3QvoT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -25.1006881,
                    "lng": 152.5572586
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.09892837010728,
                        "lng": 152.55935695
                    },
                    "southwest": {
                        "lat": -25.1016280298

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.1557732,
                    "lng": 37.2619236
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.15728587989272,
                        "lng": 37.26503565000001
                    },
                    "southwest": {
                        "lat": 25.15458622010728,
                        "lng": 37.26088624999998
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Royal Tours Permanent Camp",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/100039443836324330496\">A G

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIHXmg5JPw18EkRt4PJ6uGKyHtFkO5Dq7Oyhz0h3Li40-OpH7yxuKquBZAFuuVZ-x-bLixGRErdtUV0ScTyRBjufhz2sVFT8HMSDeM8qIS73OzVluPhEjLPbZuK-8ibjzPScnpq1VAwK9zdeoRv2snXAGYeV8MwQtEtMz9cgvgAfCn_A3ECutW8x38KQeEOTSZujVY_JAJ_Va9wm3uaVYX30qQ9c2zHPPcRFxgEr8rFiyUi4Wvy2z6j-wLw7U2qojucXwSzGlD1Nhstg_q07bOJRxljWuLE1YDuY5UVmwiKulC-r55o2HNtPtTJFbbSkE2ciDSQ-wkVH5C2AMSLEFliKeheIR-zR6Q1VMiSvXVSx6-MeO6JurZpiIQaixZ1qVXVpg-0irzkMcx_1JdbpKahCIOU-a9YMqmrJSqw1NQRod-aB1mRzipv5w0PoOOh",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.6755193,
                    "lng": 14.521949
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.67416947010728,
                        "lng": 14.52329882989272
                    },
                    "southwest": {
                        "lat": -22.6768691

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJcn8HMRCuO0yQm58CYESG1RShiEGrgXyHhVqJBEDITlgH_a-rEu5JeVF3XpPQOxWqAc78Wed8OCXxyv-65pmEjAMavLPe9rhR7omOL9kQUJliBBmXF8CDX5BAWRo78VnaB691hGavh1QBNaC36duLRh1mjb6A2xpVkMqpoz11t-TUb61VAPQT-3GsH44D-ocmCyT39DGF44cXZa9Ugl3U1TltbJrszkDFge6sLw2GhQ5eH3xHwOHm2rVczyEZyLNi31H8o8JZ2qAenBogoduRxnnwh4ZBVnm-fqzlXgUaTAMcBR97HI-OynkEP7m_p9oHr7bcDdyUgLn7E1skQGpPVAgH1g4InS4m5kXKNH_DdNHSVgzUIJ0ceMoDBmaimpp-k8PB1eyniCuiVQI0sdKHtU_lVZBLTFOaCrUj1JJiKus01_fgL3Hc1fLD__1ZY",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.14818,
                    "lng": 55.27768
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.14686767010728,
                        "lng": 55.27904012989272
                    },
                    "southwest": {
                        "lat": -21.1495673298

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"{rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))